In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
train_dir = r"D:\DP Project\Face Mask Detection\Face Mask Dataset\Train"
val_dir = r"D:\DP Project\Face Mask Detection\Face Mask Dataset\Validation"
test_dir = r"D:\DP Project\Face Mask Detection\Face Mask Dataset\Test"

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale= 1./255)


train_gen = train_datagen.flow_from_directory(train_dir,
                                             target_size=(128,128),
                                             batch_size=100,
                                             class_mode='binary')
val_gen = test_datagen.flow_from_directory(val_dir,
                                             target_size=(128,128),
                                             batch_size=16,
                                             class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [17]:
from tensorflow.keras.applications import VGG19

conv_base = VGG19(include_top = False, input_shape = (128,128,3))
from tensorflow.keras import layers
from tensorflow.keras import models

In [18]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 22,122,049
Trainable params: 22,122,049
Non-trainable params: 0
_________________________________________________________________


In [20]:
from tensorflow.keras import optimizers

model.compile(loss = 'binary_crossentropy',
             optimizer= optimizers.RMSprop(learning_rate=2e-5),
             metrics=['acc'])

In [21]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('FaceMD-{epoch:02d}.h5', save_best_only=True)

In [22]:
history = model.fit(train_gen,
                   steps_per_epoch=100,
                   epochs=10,
                   validation_data=val_gen,
                   validation_steps=50,
                   callbacks=[checkpoint_cb])

Epoch 1/10
100/100 [==============================] - 1301s 13s/step - loss: 0.1335 - acc: 0.9453 - val_loss: 0.0181 - val_acc: 0.9962
Epoch 2/10
100/100 [==============================] - 1309s 13s/step - loss: 0.0317 - acc: 0.9897 - val_loss: 0.0033 - val_acc: 0.9987
Epoch 3/10
100/100 [==============================] - 1265s 13s/step - loss: 0.0277 - acc: 0.9907 - val_loss: 0.0023 - val_acc: 0.9987
Epoch 4/10
100/100 [==============================] - 1257s 13s/step - loss: 0.0147 - acc: 0.9945 - val_loss: 0.0082 - val_acc: 0.9975
Epoch 5/10
100/100 [==============================] - 1254s 13s/step - loss: 0.0147 - acc: 0.9955 - val_loss: 0.0065 - val_acc: 0.9987
Epoch 6/10
100/100 [==============================] - 1250s 13s/step - loss: 0.0119 - acc: 0.9967 - val_loss: 0.0020 - val_acc: 0.9987
Epoch 7/10
100/100 [==============================] - 1252s 13s/step - loss: 0.0091 - acc: 0.9974 - val_loss: 6.8894e-04 - val_acc: 1.0000
Epoch 8/10
100/100 [==============================]

In [23]:
test_gen = test_datagen.flow_from_directory(test_dir,
                                             target_size=(128,128),
                                             batch_size=100,
                                             class_mode='binary')

Found 992 images belonging to 2 classes.


In [27]:
model.evaluate(test_gen, steps = 10)

10/10 [==============================] - 24s 2s/step - loss: 0.0017 - acc: 0.9990


[0.0016618533991277218, 0.9989919066429138]